# Fidelity Portfolio Review
### V 0.2.0

## What does it do?

This notebook imports data from a csv of account positions, copies the data into a new excel workbook, and fills in additional columns with additional information, such as P/E Ratios, Recognia Technical Analysis, etc.

## How does it work?

This notebook uses the Selenium Webdriver to automate Firefox to navigate through Fidelity's stock research pages and extract useful information. 

## Dependencies

- Selenium V 3.141.0
- Gecko Webdriver installed in system path
- Pandas
- Numpy


## Instructions for use



In [ ]:
# import modules
from selenium.webdriver import Firefox
import time
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import re
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from datetime import datetime
import extract
import shutil
# import yfinance

# import the csv file:

# at this time, I'm unsure if I want to continue using TK to open the csv file; last time I tried, it froze up my notebook.

# root = Tk()
# root.withdraw()
# source_data = askopenfilename()
# for now, just use the direct file path
stock_df = pd.read_csv('test_data.csv', index_col=False)
stock_df = stock_df.sort_values('Symbol')

# add the necessary columns to the df for the data we will be extracting
columns_list = ['Symbol', 'Description', 'Quantity', 'Last Price', 'Last Price Change', 'Current Value', "Today's Gain/Loss Dollar", "Today's Gain/Loss Percent", "Total Gain/Loss Dollar", "Total Gain/Loss Percent", "Percent Of Account", "Cost Basis", "Cost Basis Per Share", 'Div. Amt', 'Div. Yield', 'Equity Summary Score', 'Recognia Short Term', 'Recognia Intermediate Term', 'Recognia Long Term', 'Valuation', 'Quality', 'Growth Stability', 'Financial Health', '1-Yr Price Target', 'P/E TTM', 'P/E 5-Yr Avg', 'P/CF Recent Quarter', 'P/CF TTM', 'P/Sales Recent Quarter', 'P/Sales TTM', 'P/B', 'Debt/Equity TTM', 'Debt/Equity Recent Quarter', '52-Week Performance', "Account Name/Number"]
stock_df = stock_df.reindex(columns = columns_list)

current_time = datetime.now().strftime('%m%d%y_%H%M%S')
new_workbook = f'target_workbook_{current_time}.xlsx'

# make a copy of the target_workbook w/ the datetime in the name
# NOTE: not needed, since conditional formatting needs to be applied directly w/ openpyxl
# shutil.copy2('target_workbook.xlsx', new_workbook)

# next step: copy the existing data into the new excel document
# we will be periodically overwriting this notebook
# this will serve as backup for our data if the extract cell crashes
stock_df.to_excel(new_workbook)

# after that, open the web browser
driver = Firefox()
driver.get('https://www.fidelity.com/')
driver.execute_script('''window.open("https://finance.yahoo.com/quote/V?p=V&.tsrc=fin-srch", "_blank");''')
fidelity_window = driver.window_handles[0]
yf_window = driver.window_handles[1]
driver.switch_to.window(window_name=fidelity_window)


## Next,
Log into Fidelity's website as you normally would. Once you have logged on to Fidelity's website, you can run the data extraction cell below.

In [ ]:
# create a list of symbols to iterate over
# symbols = [i for i in stock_df['Symbol']]

# define a prev_symbol variable to save time
prev_symbol = ""

# var set to control when to save to excel
i = 0

# iterate thru the symbols and write to the DF
for row in stock_df.itertuples():
    
    symbol = row[1]

    # check to see if we already have data for the symbol
    if symbol == prev_symbol:
        # unsure if this will work
        prev_series = stock_df.iloc[(i - 1), 13:34]
        stock_df.iloc[i, 13:34] = prev_series
        i += 1
    else:
        
        # load the page, and scroll thru it
        driver.get(f"https://snapshot.fidelity.com/fidresearch/snapshot/landing.jhtml#/research?symbol={symbol}&appCode=&optInNRE=")
        time.sleep(5)   # pauses may need to be adjusted based on network connection
        driver.execute_script("window.scrollTo(0, 0)")
        time.sleep(3)
        driver.execute_script("window.scrollTo(0,(document.body.scrollHeight / 2))")
        time.sleep(3)
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

        # now, check the security type
        sec_type = extract.determine_security_type(driver)
        if sec_type == 'Stocks':

            # Dividends
            dividends = extract.stock_div_yield(driver)
            stock_df.at[row.Index, 'Div. Amt'] = dividends[0]
            stock_df.at[row.Index, 'Div. Yield'] = dividends[1]

            # Equity Summary Score
            equity_sum_score = extract.stock_equity_sum_score(driver)
            stock_df.at[row.Index, 'Equity Summary Score'] = equity_sum_score

            # Recognia Technical Analysis
            recognia_dict = extract.recognia_analysis(driver, sec_type)
            stock_df.loc[row.Index, 'Recognia Short Term'] = recognia_dict['Recognia Short Term']
            stock_df.loc[row.Index, 'Recognia Intermediate Term'] = recognia_dict['Recognia Intermediate Term']
            stock_df.loc[row.Index, 'Recognia Long Term'] = recognia_dict['Recognia Long Term']

            # Stock Valuation
            valuation = extract.stock_valuation(driver)
            stock_df.at[row.Index, 'Valuation'] = valuation

            # Stock Quality
            quality = extract.stock_quality(driver)
            stock_df.at[row.Index, 'Quality'] = quality

            # Stock Growth Stability
            growth_stability = extract.stock_growth_stability(driver)
            stock_df.at[row.Index, 'Growth Stability'] = growth_stability

            # Stock Financial Health
            financial_health = extract.stock_financial_health(driver)
            stock_df.at[row.Index, 'Financial Health'] = financial_health

            # 52-Week Performance
            ftw_perf = extract.stock_growth_stability(driver)
            stock_df.at[row.Index, '52-Week Performance'] = ftw_perf

            time.sleep(1)

            # Key Stats
            # I'm honestly not sure if this will work
            key_stats = extract.stock_key_stats(driver, symbol)
            try: 
                stock_df.at[row.Index, 'P/E TTM'] = key_stats[0]['P/E (Trailing Twelve Months)']
            except:
                stock_df.loc[row.Index, 'P/E TTM'] = np.nan
            try:
                stock_df.at[row.Index, 'P/E 5-Yr Avg'] = key_stats[0]['P/E (5-Year Average)']
            except:
                stock_df.loc[row.Index, 'P/E 5-Yr Avg'] = np.nan
            try:
                stock_df.at[row.Index, 'P/CF Recent Quarter'] = key_stats[0]['Price/Cash Flow (Most Recent Quarter)']
            except:
                stock_df.loc[row.Index, 'P/CF Recent Quarter'] = np.nan
            try:
                stock_df.at[row.Index, 'P/CF TTM'] = key_stats[0]['Price/Cash Flow (TTM)']
            except:
                stock_df.loc[row.Index, 'P/CF TTM'] = np.nan
            try:
                stock_df.at[row.Index, 'P/Sales Recent Quarter'] = key_stats[0]['Price/Cash Flow (TTM)']
            except:
                stock_df.loc[row.Index, 'P/Sales Recent Quarter'] = np.nan
            try:
                stock_df.at[row.Index, 'P/Sales TTM'] = key_stats[0]['Price/Sales (TTM)']
            except:
                stock_df.loc[row.Index, 'P/Sales TTM'] = np.nan
            try:
                stock_df.at[row.Index, 'P/B'] = key_stats[0]['Price/Book']
            except:
                stock_df.loc[row.Index, 'P/B'] = np.nan
            try:
                stock_df.at[row.Index, 'Debt/Equity TTM'] = key_stats[1]['Total Debt/Equity (Most Recent Quarter, Annualized)']
            except:
                stock_df.loc[row.Index, 'Debt/Equity TTM'] = np.nan
            try:
                stock_df.at[row.Index, 'Debt/Equity Recent Quarter'] = key_stats[1]['Total Debt/Equity (TTM)']
            except:
                stock_df.loc[row.Index, 'Debt/Equity Recent Quarter'] = np.nan

            time.sleep(1)

            # 1-Yr Price Target
            price_target = extract.stock_one_yr_price_target(driver, symbol)
            stock_df.at[row.Index, '1-Yr Price Target'] = price_target

        elif sec_type == 'ETFs':
            
            # ETF Stats
            etf_stats = extract.etf_key_stats(driver)
            try:
                stock_df.at[row.Index, 'P/E TTM'] = etf_stats['Price / Earnings (TTM)']
            except:
                stock_df.loc[row.Index, 'P/E TTM'] = np.nan
            try:
                stock_df.at[row.Index, 'P/B'] = etf_stats['Price / Book']
            except:
                stock_df.loc[row.Index, 'P/B'] = np.nan
            try:
                stock_df.at[row.Index, 'P/Sales TTM'] = etf_stats['Price / Sales']
            except:
                stock_df.loc[row.Index, 'P/Sales TTM'] = np.nan
            try:
                stock_df.at[row.Index, 'P/CF TTM'] = etf_stats['Price / Cash Flow']
            except:
                stock_df.loc[row.Index, 'P/CF TTM'] = np.nan
            try:
                stock_df.at[row.Index, 'Div. Yield'] = etf_stats['Distribution Yield']
            except:
                stock_df.loc[row.Index, 'Div. Yield'] = np.nan

            # ETF 52 Week Performance
            ftw_perf = extract.etf_52_week_perf(driver)
            stock_df.at[row.Index, '52-Week Performance'] = ftw_perf

        elif sec_type == 'Mutual Funds':

            # 52 Week Performance
            ftw_perf = extract.mf_52_week_perf(driver)
            stock_df.at[row.Index, '52-Week Performance'] = ftw_perf

        i += 1
        if (i % 5) == 0:
            stock_df.to_excel(new_workbook)
            print('DF saved to Excel File')
        prev_symbol = symbol
        # prev_series = stock_df.iloc[row.Index, 13:34]
        prev_row = row.Index

# EVERYTHING BELOW THIS CELL IS TO BE DELETED BEFORE THIS NOTEBOOK IS FINISHED

In [ ]:
# test; delete when finished
for row in stock_df.itertuples():
    print(row.Index)

In [ ]:
 # test cell
 stock_df.loc[100, 'Cost Basis Per Share']

In [ ]:
# test cell
test_dict = {
    'test': 50
}

other_dict = {
    1: test_dict
}
other_dict[1]['test']

In [ ]:
# test cell
# use df.at for strings?
stock_df.at[65, 'Equity Summary Score'] = 0
stock_df.loc[65, 'Recognia Short Term'] = 'weak'

In [ ]:
stock_df.iloc[49, 13:34]

In [ ]:
stock_df[stock_df['Symbol'] == 'AAPL'].index

In [ ]:
stock_df.iloc[stock_df[stock_df['Symbol'] == 'AAPL'].index[0], 13:34]

In [ ]:
stock_df.iloc[2,13:34] = stock_df.iloc[0, 13:34]